In [16]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [17]:
base = pd.read_csv('iris.csv')
base.head(3)

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [18]:
previsores = base.iloc[:, 0:4].values
previsores[0:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [19]:
classe = base.iloc[:, 4].values
classe[0:3]

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa'], dtype=object)

In [20]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
classe = labelencoder.fit_transform(classe)
classe[0:3]

array([0, 0, 0])

In [21]:
import gc
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import Callback

In [22]:
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

In [23]:
def criar_rede():
    # cleanup
    k.clear_session()
    
    classificador = Sequential()
    
    classificador.add(Dense(units = 8, activation = 'relu', 
                        kernel_initializer = 'normal', input_dim = 4))
    
    classificador.add(Dropout(0.2))
    
    classificador.add(Dense(units = 8, activation = 'relu', 
                        kernel_initializer = 'random_uniform'))
    
    classificador.add(Dropout(0.3))
    
    classificador.add(Dense(units = 3, activation = 'softmax'))
    
    classificador.compile(optimizer = 'RMSprop', loss = 'categorical_crossentropy',
                          metrics = ['categorical_accuracy'])
    
    return classificador

In [24]:
classificador = KerasClassifier(build_fn = criar_rede,
                                epochs = 500,
                                batch_size = 10)
                                #callbacks= ClearMemory() )

In [25]:
from sklearn.utils import parallel_backend
import time

In [26]:
start = time.time()
with parallel_backend('threading'):
    resultados = cross_val_score(estimator = classificador,
                         X = previsores, y = classe,
                         cv = 5, scoring = 'f1_weighted',
                         n_jobs = 1)

Epoch 1/500
12/12 [==============================] - 0s 999us/step - loss: 1.0735 - categorical_accuracy: 0.4250
Epoch 2/500
12/12 [==============================] - 0s 1ms/step - loss: 1.0502 - categorical_accuracy: 0.4750
Epoch 3/500
12/12 [==============================] - 0s 999us/step - loss: 1.0288 - categorical_accuracy: 0.4000
Epoch 4/500
12/12 [==============================] - 0s 916us/step - loss: 1.0077 - categorical_accuracy: 0.3917
Epoch 5/500
12/12 [==============================] - 0s 1ms/step - loss: 1.0013 - categorical_accuracy: 0.4667
Epoch 6/500
12/12 [==============================] - 0s 916us/step - loss: 0.9810 - categorical_accuracy: 0.4167
Epoch 7/500
12/12 [==============================] - 0s 999us/step - loss: 0.9901 - categorical_accuracy: 0.4083
Epoch 8/500
12/12 [==============================] - 0s 916us/step - loss: 0.9798 - categorical_accuracy: 0.4167
Epoch 9/500
12/12 [==============================] - 0s 916us/step - loss: 1.0056 - categorical_accu

In [27]:
end = time.time()
total = (end - start)/60
print('{:.2f}min'.format(total))

0.72min


In [28]:
resultados

array([1.        , 1.        , 0.90909091, 1.        , 0.90909091])

In [29]:
media = resultados.mean()
desvio = resultados.std()

In [30]:
print('accuracy mean: {}'.format(media))
print('accuracy Desvio: {}'.format(desvio))

accuracy mean: 0.9636363636363636
accuracy Desvio: 0.04453617714151235
